# Import Libraries

In [ ]:
#DataFrames manipulation
import pandas as pd
import numpy as np

#Date manipulation
import datetime

#Data download from Yahoo Finance
from yahooquery import Ticker

#Libraries for the Plotting
import holoviews as hv
from holoviews import opts, dim
from holoviews.plotting.links import RangeToolLink
from bokeh.models import HoverTool
from bokeh.palettes import Category10

#Librarie to save the plots to html object
import panel as pn
import param

hv.extension('bokeh')

# Download Data from Yahoo Finance

In [ ]:
#Download the Data from Yahoo Finance
tickers = Ticker('^SPX')
df = tickers.history(start = '1970-01-01')

#Limit the Data to the Close and rearrange columns
df = df.reset_index()
df = df[['date', 'close']]
df = df.rename(columns = {'date': 'Date', 'close': 'S&P500'})
df = df.set_index('Date')

#Calculate the Percentage Change
df = df.pct_change()

#Change the Data to Index format base = 100
df.iloc[0] = 0
df['S&P500'] = (1 + df['S&P500']).cumprod() * 100

# Define Bull and Bear Markets

In [ ]:
# Define the Threshold Percentages for Bull and Bear Markets
bull_threshold = 0.2
bear_threshold = -0.2

#CreateNnew columns for Previous Top and Previous Vottom
df['Previous Top'] = df['S&P500'].copy()
df['Previous Bottom'] = df['S&P500'].copy()

#Create a New column for Market Type - Initially filled with 'None'
df['Market'] = None

#Initialize variables to track PreviousTtop and Previous Bottom
prev_top = df['S&P500'].iloc[0]
prev_bottom = df['S&P500'].iloc[0]
current_market = None

#Loop through the DataFrame to identify Bull and Bear Markets
for i, row in df.iterrows():
    if current_market is None:
        if row['S&P500'] >= prev_bottom * (1 + bull_threshold):
            current_market = 'Bull'
            df.at[i, 'Market'] = current_market
        elif row['S&P500'] <= prev_top * (1 + bear_threshold):
            current_market = 'Bear'
            df.at[i, 'Market'] = current_market
    elif current_market == 'Bull':
        if row['S&P500'] <= prev_top * (1 + bear_threshold):
            current_market = 'Bear'
            df.at[i, 'Market'] = current_market
            prev_bottom = row['S&P500']
        else:
            df.at[i, 'Market'] = current_market
            prev_top = max(prev_top, row['S&P500'])
    elif current_market == 'Bear':
        if row['S&P500'] >= prev_bottom * (1 + bull_threshold):
            current_market = 'Bull'
            df.at[i, 'Market'] = current_market
            prev_top = row['S&P500']
        else:
            df.at[i, 'Market'] = current_market
            prev_bottom = min(prev_bottom, row['S&P500'])
    
    df.at[i, 'Previous Top'] = prev_top
    df.at[i, 'Previous Bottom'] = prev_bottom

# Create the Chart of the S&P 500 Bull and Bear Markets

In [ ]:
#Drop rows with None values in 'Market' column
df = df.dropna(subset=['Market'])

#Rename the column S&P 500 not to have a symbol
df = df.rename(columns = {'S&P500' : 'SP500'})

#Create a HoloViews Curve for the S&P500 values
hover = HoverTool(tooltips=[("Date", "@Date{%F}"), ('SP500', f"@{'SP500{0.00}'}")], formatters={'@Date': 'datetime'})
spx_curve = hv.Curve(df, 'Date', 'SP500').opts(line_color='white', show_legend=False, tools=[hover])

#Create a HoloViews Scatter for the Market Type
market_scatter = hv.Scatter(df, 'Date', [('SP500', 'SP500'), ('Market', 'Market')]).opts(legend_position='top_left')

#Define the Options for the Chart
opts.defaults(opts.Curve(width=800, height=400),
              opts.Scatter(color='Market', cmap={'Bear': 'red', 'Bull': 'green'}))

#Define the Distance (in Years) between ticks
distance_ticks = 5
number_ticks = int(round(((df.index[-1].year - df.index[0].year) / distance_ticks), 0))

#Combine the Curves and the Scatter Plot
full_graph = spx_curve * market_scatter

# Set the y-axis label for the 'S&P500' values
full_graph = full_graph.options(opts.Curve(ylabel='S&P 500® Index', xticks = number_ticks,
                                           title = 'S&P 500® Index: History of Bull and Bear Markets'))

In [ ]:
#Save graph to Html
p = pn.panel(full_graph)
p.save('SP500_Bull_Bear_Market_History_Graph.html', embed = True)

# Table with Each Bull and Bear Market

In [ ]:
#Rename the column S&P 500 back
df = df.rename(columns = {'SP500' : 'S&P500'})

# Create an empty list to store market cycles
market_cycles = []

# Initialize variables to track market cycle information
current_market_type = None
start_date = None
end_date = None

# Iterate through the 'Market' column
for date, market_type in df['Market'].iteritems():
    if market_type != current_market_type:
        # A new market cycle has started
        if current_market_type is not None:
            # Add the previous market cycle to the list
            cycle_length = (end_date - start_date).days + 1
            start_value = df.loc[start_date, 'S&P500']
            end_value = df.loc[end_date, 'S&P500']
            sp500_top = df.loc[start_date:end_date, 'S&P500'].max()
            sp500_bottom = df.loc[start_date:end_date, 'S&P500'].min()
            market_cycles.append({
                'Market Type': current_market_type,
                'Start Date': start_date,
                'End Date': end_date,
                'Length (Days)': cycle_length,
                'S&P 500 at Start Date': start_value,
                'S&P 500 at End Date': end_value,
                'S&P 500 Top': sp500_top,
                'S&P 500 Bottom': sp500_bottom
            })

        # Update current market type and start date
        current_market_type = market_type
        start_date = date

    # Update end date
    end_date = date

# Create the market_cycles_df dataframe
market_cycles_df = pd.DataFrame(market_cycles)

# Set the 'Start Date' and 'End Date' columns as datetime
market_cycles_df['Start Date'] = pd.to_datetime(market_cycles_df['Start Date'])
market_cycles_df['End Date'] = pd.to_datetime(market_cycles_df['End Date'])

In [ ]:
#Adding the Market Return, Maximum Possible Profit and Maximum Possible Loss
market_cycles_df['Market Return (%)'] = round((market_cycles_df['S&P 500 at End Date'] / market_cycles_df['S&P 500 at Start Date'] - 1) * 100, 2)
market_cycles_df['Max. Poss. Profit (%)'] = round((market_cycles_df['S&P 500 Top'] / market_cycles_df['S&P 500 at Start Date'] - 1) * 100, 2)
market_cycles_df['Max. Poss. Loss (%)'] = round((market_cycles_df['S&P 500 Bottom'] / market_cycles_df['S&P 500 at Start Date'] - 1) * 100, 2)

In [ ]:
#Let's create a New Dataframe with only the Statistics, which is what we want to show
market_cycles_stats_df = market_cycles_df[['Market Type', 'Start Date', 'End Date', 'Length (Days)', 'Market Return (%)',
                                          'Max. Poss. Profit (%)', 'Max. Poss. Loss (%)']]

# Convert the 'Start Date' and 'End Date' columns to datetime
market_cycles_stats_df['Start Date'] = pd.to_datetime(market_cycles_stats_df['Start Date'])
market_cycles_stats_df['End Date'] = pd.to_datetime(market_cycles_stats_df['End Date'])

# Format the 'Start Date' and 'End Date' columns to display only the date part
market_cycles_stats_df['Start Date'] = market_cycles_stats_df['Start Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
market_cycles_stats_df['End Date'] = market_cycles_stats_df['End Date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [ ]:
#Create Holoviews table object for the Market Cycle Statistics Table
market_cycles_stats_table = hv.Table(market_cycles_stats_df).opts(
    opts.Table(width=950, height=350, selectable = True, index_position = None, 
               title = 'S&P 500® Index: Bull and Bear Market Statistics'))

In [ ]:
#Save the Table
p = pn.panel(market_cycles_stats_table)
p.save('Market_Cycle_Stats_Table.html', embed = True)

# Table with Bull Markets and Forward Returns

In [ ]:
#Instatiate a New Object for the Bull Market Cycles
bull_cycles_perf_df = market_cycles_df[['Market Type', 'Start Date', 'End Date', 'Length (Days)', 'S&P 500 at Start Date']]
bull_cycles_perf_df = bull_cycles_perf_df[bull_cycles_perf_df['Market Type'] == 'Bull']

In [ ]:
#Define the Time Intervals for Performance Measures
one_week = datetime.timedelta(weeks=1)
one_month = datetime.timedelta(days=30)
three_months = datetime.timedelta(days=90)
six_months = datetime.timedelta(days=180)
one_year = datetime.timedelta(days=365)

#Create the Performance Columns with Initial NaN Values
bull_cycles_perf_df['1 Week'] = np.nan
bull_cycles_perf_df['1 Month'] = np.nan
bull_cycles_perf_df['3 Months'] = np.nan
bull_cycles_perf_df['6 Months'] = np.nan
bull_cycles_perf_df['1 Year'] = np.nan

#Iterate over the rows of bull_cycles_perf_df dataframe to Calculate Performance Measures
for index, row in bull_cycles_perf_df.iterrows():
    start_date = row['Start Date']
    end_date_1w = start_date + one_week
    end_date_1m = start_date + one_month
    end_date_3m = start_date + three_months
    end_date_6m = start_date + six_months
    end_date_1y = start_date + one_year
    
    #Calculate Performance Measures based on Corresponding Values in df
    value_1w = df.loc[start_date:end_date_1w, 'S&P500'].iloc[-1]
    value_1m = df.loc[start_date:end_date_1m, 'S&P500'].iloc[-1]
    value_3m = df.loc[start_date:end_date_3m, 'S&P500'].iloc[-1]
    value_6m = df.loc[start_date:end_date_6m, 'S&P500'].iloc[-1]
    value_1y = df.loc[start_date:end_date_1y, 'S&P500'].iloc[-1]
    
    #Assign the Calculated Performance Measures to the Corresponding Columns
    bull_cycles_perf_df.at[index, '1 Week'] = value_1w
    bull_cycles_perf_df.at[index, '1 Month'] = value_1m
    bull_cycles_perf_df.at[index, '3 Months'] = value_3m
    bull_cycles_perf_df.at[index, '6 Months'] = value_6m
    bull_cycles_perf_df.at[index, '1 Year'] = value_1y

In [ ]:
#Calculate the Performance for every colum
bull_cycles_perf_df['1 Week'] = round((bull_cycles_perf_df['1 Week'] / bull_cycles_perf_df['S&P 500 at Start Date'] - 1) * 100, 2)
bull_cycles_perf_df['1 Month'] = round((bull_cycles_perf_df['1 Month'] / bull_cycles_perf_df['S&P 500 at Start Date'] - 1) * 100, 2)
bull_cycles_perf_df['3 Months'] = round((bull_cycles_perf_df['3 Months'] / bull_cycles_perf_df['S&P 500 at Start Date'] - 1) * 100, 2)
bull_cycles_perf_df['6 Months'] = round((bull_cycles_perf_df['6 Months'] / bull_cycles_perf_df['S&P 500 at Start Date'] - 1) * 100, 2)
bull_cycles_perf_df['1 Year'] = round((bull_cycles_perf_df['1 Year'] / bull_cycles_perf_df['S&P 500 at Start Date'] - 1) * 100, 2)

In [ ]:
#Drop S&P 500 at Start Date column - No longer needed
bull_cycles_perf_df = bull_cycles_perf_df.drop(columns = 'S&P 500 at Start Date')

#Convert the 'Start Date' and 'End Date' columns to String for the table
bull_cycles_perf_df['Start Date'] = pd.to_datetime(bull_cycles_perf_df['Start Date']).dt.strftime('%Y-%m-%d')
bull_cycles_perf_df['End Date'] = pd.to_datetime(bull_cycles_perf_df['End Date']).dt.strftime('%Y-%m-%d')

In [ ]:
#Calculate the Average Values for the Performance Columns
average_row = round(bull_cycles_perf_df.mean(numeric_only=True), 2)

#Create a Dictionary for the Average Row
average_dict = {
    'Market Type': 'Bull',
    'Start Date': 'Average',
    'End Date': '-',
    'Length (Days)': round(average_row['Length (Days)'],0),
    '1 Week': average_row['1 Week'],
    '1 Month': average_row['1 Month'],
    '3 Months': average_row['3 Months'],
    '6 Months': average_row['6 Months'],
    '1 Year': average_row['1 Year']
}

#Append the Average Row to the DataFrame
bull_cycles_perf_df = bull_cycles_perf_df.append(average_dict, ignore_index=True)

In [ ]:
#Create Holoviews table object for the Bull Market Performance
bull_market_perf_table = hv.Table(bull_cycles_perf_df).opts(
    opts.Table(width=950, height=290, selectable = True, index_position = None, 
               title = 'S&P 500® Index: Future Performance after entering a Bull Market'))

In [ ]:
#Save the Table
p = pn.panel(bull_market_perf_table)
p.save('Bull_Market_Perf_Table.html', embed = True)